In [ ]:
import pathlib

import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = "inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,
    weights=None
)

pre_trained_model.load_weights(local_weights_file)

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer("mixed7")
last_output = last_layer.output

print(last_layer, last_output)

In [ ]:
from tensorflow.keras import layers

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

In [ ]:
from tensorflow.keras import Model

model = Model( pre_trained_model.input, x )

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss=tf.keras.losses.KLDivergence(),
    metrics=["accuracy", tf.keras.metrics.F1Score()]
)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0
)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0
)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255
)

In [ ]:
data_dir = pathlib.Path("Cats_Dogs_Mini/")

In [ ]:
target_size = (150, 150)
batch_size = 100

train_generator = train_datagen.flow_from_directory(
    data_dir.joinpath("train"),
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    data_dir.joinpath("validation"),
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    data_dir.joinpath("test"),
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary'
)

In [ ]:
model.fit(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=20,
    epochs=10,
    validation_steps=10
)